In [78]:
import pandas as pd 

In [79]:
ds = pd.read_csv('Syncadia_Data.csv')

In [112]:
ds.head()

title  tag     artist  year   views  \
0          Killa Cam  rap    Cam'ron  2004  173166   
1         Can I Live  rap      JAY-Z  1996  468624   
2  Forgive Me Father  rap   Fabolous  2003    4743   
3       Down and Out  rap    Cam'ron  2004  144404   
4             Fly In  rap  Lil Wayne  2005   78271   

                                       features  \
0                   {"Cam\\'ron","Opera Steve"}   
1                                            {}   
2                                            {}   
3  {"Cam\\'ron","Kanye West","Syleena Johnson"}   
4                                            {}   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [81]:
!pip install transformers

python(61546) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [116]:
custom_label_mapping = {
    "admiration": "admiration",   # Retain as-is
    "amusement": "joy",           # Merge with joy
    "anger": "anger",             # Retain as-is
    "annoyance": "anger",         # Merge with anger
    "approval": "admiration",     # Merge with admiration
    "caring": "love",             # Merge with love
    "confusion": "confusion",     # Retain as-is
    "curiosity": "curiosity",     # Retain as-is
    "desire": "desire",           # Retain as-is
    "disappointment": "sadness",  # Merge with sadness
    "disapproval": "anger",       # Merge with anger
    "disgust": None,              # Remove
    "embarrassment": None,        # Remove
    "excitement": "joy",          # Merge with joy
    "fear": "fear",               # Retain as-is
    "gratitude": "admiration",    # Merge with admiration
    "grief": "sadness",           # Merge with sadness
    "joy": "joy",                 # Retain as-is
    "love": "love",               # Retain as-is
    "nervousness": "fear",        # Merge with fear
    "optimism": "joy",            # Merge with joy
    "pride": "admiration",        # Merge with admiration
    "realization": None,          # Remove
    "relief": "joy",              # Merge with joy
    "remorse": "sadness",         # Merge with sadness
    "sadness": "sadness",         # Retain as-is
    "surprise": "surprise",       # Retain as-is
    "neutral": "neutral"          # Retain as fallback
}

In [139]:
# Sample of 100 rows for testing
ds_sample = ds.sample(n=1000, random_state=42)

In [152]:
# Sentiment classifier
classifier = pipeline(
    "text-classification",
    model="SamLowe/roberta-base-go_emotions",
    top_k=None,  # Replaces return_all_scores=True
    device=device  # Use GPU if available
)
from tqdm import tqdm

# Function to analyze song sentiment and return top sentiments
def analyze_song_sentiment(lyrics, top_n=3):
    truncated_lyrics = lyrics[:812] if isinstance(lyrics, str) and len(lyrics) > 812 else lyrics
    predictions = classifier(truncated_lyrics)[0]
    custom_predictions = [
        {"label": custom_label_mapping[pred["label"]], "score": pred["score"]}
        for pred in predictions if custom_label_mapping.get(pred["label"])
    ]
    total_score = sum(pred["score"] for pred in custom_predictions)
    for pred in custom_predictions:
        pred["score"] /= total_score  # Normalize scores
    return sorted(custom_predictions, key=lambda x: x["score"], reverse=True)[:top_n]

# Recompute 'top_sentiments'
tqdm.pandas()
ds_sample['top_sentiments'] = ds_sample['lyrics'].progress_apply(lambda x: analyze_song_sentiment(x, top_n=3))

100%|███████████████████████████████████████| 1000/1000 [05:29<00:00,  3.04it/s]


In [142]:
ds_sample.head()

,title,tag,artist,year,views,features,lyrics,language,top_sentiments
229913,Until I Fall Away,rock,Gin Blossoms,1992,11257,{},[Verse 1]\nI want to tell if I am or I am not myself\nIt's hard to know how far or if at all could go\nWaiting far too long for something I forgot was wrong\nI don't know all the answers I think that I'll find\nOr have it within the time but it's all that I'll have in mind\n\n[Chorus]\nUntil I fall away\nThat won't keep us waiting long\nUntil I fall away\nI don't know what to do anymore\nUntil I fall away\n\n[Verse 2]\nMy fear pretend that I'll never be in love again\nIt's real to me but not like these fools and not like this scene\nI won't find or have it within the time\nIf it's all rusted and fade in the spot where we fell\nWhere I thought I'd left behind\nIt's loose now but we could try\n[Chorus]\nWhen there's no good answers\nTo those new questions\nAnother personal disaster\nThere's nowhere to go but down,en,"[{'label': 'confusion', 'score': 0.3406164521002825}, {'label': 'sadness', 'score': 0.19732452538029605}, {'label': 'neutral', 'score': 0.1937919501045029}]"
178699,Drag Open,rock,Women,2010,2128,{},[Verse 1]\nComing too far\nAnd dragging vice\nYour lungs expand\nSo don't feel blue\n\n[Chorus]\nShe did out your voice and lips\nYou're such a drag open\n\n[Verse 2]\nSit on your back\nWe'll see this through\nYour chartered trips\nYou're smaller than sand\n\n[Chorus]\nShe did out your voice and lips\nYou're such a drag open,en,"[{'label': 'neutral', 'score': 0.932255245221613}, {'label': 'admiration', 'score': 0.014204346628701231}, {'label': 'anger', 'score': 0.012642718006340908}]"
167613,Great Awakening,rock,Rise Against,2001,1411,{},Buried words\nUnder lights\nSoundtrack to this stagnant life\nThis meaning lost in translation\nMessage sucked out of his hands again\n\nThis could be my great awakening\nBut how would I know\nCuz it's all noise to me\nAre these words falling on deaf ears?\n\nStaring at the empty stares\nRealize that he no longer cares about the wrong or right\nThe downward spiral begins tonight\n\nAnd this could be my great awakening\nBut how would I know cuz it's all noise to me\nAre these words falling on deaf ears?\n\nAnd this could be my great awakening\nBut how would I know cuz it's all noise to me\nNothing I can do will make you hear,en,"[{'label': 'neutral', 'score': 0.3012570877334345}, {'label': 'confusion', 'score': 0.22377696832894087}, {'label': 'surprise', 'score': 0.18164260772861446}]"
61924,Worthy,rb,Jacob Banks,2014,56091,{},"[Verse 1]\nSuddenly the clouds have opened up\nAnd I'm being seen in all of my glory\nThe time has come for fate to decide\nIf a part of me is even worthy\nThe air is warm, my heart is cold\nAnd I'll never know how it feels\nThe air is warm, my heart is cold\nAnd I'll never know how it feels\nTo have a heart of gold\n\n[Chorus]\nI feel worthy somehow\nPlease, I don't want to go down, down\nThe new world is calling me now\nPlease don't let me go down, down\nAnd I feel worthy somehow\nPlease don't let me go down, down\nThe new world is calling me now\nPlease don't let me go down, down\n[Verse 2]\nJust one step unto the lonely road\nHe scarred me for life\nNo time to reflect as I was getting by\nGetting by, enjoying life\nThe time has come for judgment day\nAnd I'm standing here, begging you\nPlease take pity on me, take pity on me\n\n[Chorus]\n\n[Outro]\nOh, please don't let me go down\nI don't want to go down",en,"[{'label': 'neutral', 'score': 0.376521825651974}, {'label': 'desire', 'score': 0.2036650365329073}, {'label': 'sadness', 'score': 0.1566506865188695}]"
128751,The Love Song of Saint Sebastian,misc,T.S. Eliot,1996,6914,{},"I would come in a shirt of hair\nI would come with a lamp in the night\nAnd sit at the foot of your stair;\nI would flog myself until I bled,\nAnd after hour on hour of prayer\nAnd torture and delight\nUntil my blood should ring the lamp\nAnd glisten in the light;\nI should arise your neophyte\nAnd then put out th

In [153]:
# Generate 'dominant_sentiment' from 'top_sentiments'
ds_sample['dominant_sentiment'] = ds_sample['top_sentiments'].apply(
    lambda x: x[0]['label'] if isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict) and 'label' in x[0] else None
)

# Verify the output
print(ds_sample[['top_sentiments', 'dominant_sentiment']].head())

                                                                                                                                                   top_sentiments  \
0    [{'label': 'confusion', 'score': 0.3406164521002825}, {'label': 'sadness', 'score': 0.19732452538029605}, {'label': 'neutral', 'score': 0.1937919501045029}]   
1   [{'label': 'neutral', 'score': 0.932255245221613}, {'label': 'admiration', 'score': 0.014204346628701231}, {'label': 'anger', 'score': 0.012642718006340908}]   
2  [{'label': 'neutral', 'score': 0.3012570877334345}, {'label': 'confusion', 'score': 0.22377696832894087}, {'label': 'surprise', 'score': 0.18164260772861446}]   
3         [{'label': 'neutral', 'score': 0.376521825651974}, {'label': 'desire', 'score': 0.2036650365329073}, {'label': 'sadness', 'score': 0.1566506865188695}]   
4        [{'label': 'neutral', 'score': 0.8386157487412785}, {'label': 'anger', 'score': 0.034018412280106616}, {'label': 'fear', 'score': 0.029909453924778513}]   

  dominan

In [154]:
# DataFrame to ds_sample.csv
ds_sample.to_csv("ds_sample.csv", index=False)

print("Results saved to ds_sample.csv")

Results saved to ds_sample.csv


In [155]:
print(ds_sample.columns)

Index(['title', 'tag', 'artist', 'year', 'views', 'features', 'lyrics',
       'language', 'top_sentiments', 'dominant_sentiment'],
      dtype='object')


In [156]:
# Correct required_columns to match the actual column names in ds_sample
required_columns = ['title', 'artist', 'year', 'dominant_sentiment', 'lyrics', 'top_sentiments','tag','views','features','language']

# Validation step
if not all(col in ds_sample.columns for col in required_columns):
    raise ValueError(f"Missing required columns in ds_sample.csv. Expected columns: {required_columns}")

print("Models and data loaded successfully!")

Models and data loaded successfully!


In [162]:
from flask import Flask, request, jsonify
from transformers import BlipProcessor, BlipForConditionalGeneration, pipeline
from PIL import Image
import pandas as pd
import torch

# If using flask 
"""# Initialize Flask app 
 app = Flask(__name__)"""

# Models and preprocessed song data
print("Loading models and data...")
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")
classifier = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None, device=0)
ds_sample = pd.read_csv("ds_sample.csv")  # Ensure the CSV contains the preprocessed 'dominant_sentiment' column
print("Models and data loaded successfully!")

# Definiing modular pipeline functions
def generate_caption(image, max_new_tokens=50):
    inputs = blip_processor(image, return_tensors="pt")
    with torch.no_grad():
        output = blip_model.generate(**inputs, max_new_tokens=max_new_tokens)
    return blip_processor.decode(output[0], skip_special_tokens=True)

def analyze_caption_sentiment(caption, top_n=3):
    predictions = classifier(caption)[0]
    custom_predictions = [
        {"label": custom_label_mapping[pred["label"]], "score": pred["score"]}
        for pred in predictions if custom_label_mapping.get(pred["label"])
    ]
    total_score = sum(pred["score"] for pred in custom_predictions)
    for pred in custom_predictions:
        pred["score"] /= total_score
    return sorted(custom_predictions, key=lambda x: x["score"], reverse=True)[:top_n]

import random

def find_matching_song(image_sentiment, ds_sample, top_n=3):
    # Get the top sentiments from image_sentiment
    top_sentiments = [sent["label"] for sent in image_sentiment[:top_n]]

    # Filter songs with matching sentiments
    matching_songs = ds_sample[ds_sample['dominant_sentiment'].isin(top_sentiments)].copy()

    if not matching_songs.empty:
        # Assign a match score based on top_sentiments ranking
        matching_songs['match_score'] = matching_songs['dominant_sentiment'].apply(
            lambda x: top_sentiments.index(x) if x in top_sentiments else len(top_sentiments)
        )
        
        # Randomly choose one song, with better scores more likely to be chosen
        return matching_songs.sample(n=1, weights=1 / (matching_songs['match_score'] + 1))
    
    return None

def image_to_song_pipeline(image, ds_sample, max_new_tokens=50, top_n=3):
    caption = generate_caption(image, max_new_tokens)
    image_sentiment = analyze_caption_sentiment(caption, top_n)
    matching_song = find_matching_song(image_sentiment, ds_sample)
    return caption, image_sentiment, matching_song

"""# Define the API endpoint
@app.route('/recommend', methods=['POST'])
def recommend_song():
    try:
        # Get uploaded image
        if 'image' not in request.files:
            return jsonify({"error": "No image uploaded"}), 400
        
        file = request.files['image']
        image = Image.open(file)
        
        # Process image using the pipeline
        caption, image_sentiment, matching_song = image_to_song_pipeline(image, ds_sample)
        
        # Prepare and return the response
        response = {
            "caption": caption,
            "image_sentiment": image_sentiment,
            "song": matching_song[['title', 'artist', 'year', 'dominant_sentiment']].to_dict(orient='records') if matching_song is not None else None
        }
        return jsonify(response)
    
    except Exception as e:
        print("Error:", e)
        return jsonify({"error": str(e)}), 500

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True, host="0.0.0.0", port=5000)"""


Loading models and data...
Models and data loaded successfully!


'# Define the API endpoint\n@app.route(\'/recommend\', methods=[\'POST\'])\ndef recommend_song():\n    try:\n        # Get uploaded image\n        if \'image\' not in request.files:\n            return jsonify({"error": "No image uploaded"}), 400\n        \n        file = request.files[\'image\']\n        image = Image.open(file)\n        \n        # Process image using the pipeline\n        caption, image_sentiment, matching_song = image_to_song_pipeline(image, ds_sample)\n        \n        # Prepare and return the response\n        response = {\n            "caption": caption,\n            "image_sentiment": image_sentiment,\n            "song": matching_song[[\'title\', \'artist\', \'year\', \'dominant_sentiment\']].to_dict(orient=\'records\') if matching_song is not None else None\n        }\n        return jsonify(response)\n    \n    except Exception as e:\n        print("Error:", e)\n        return jsonify({"error": str(e)}), 500\n\n# Run the Flask app\nif __name__ == \'__main__

In [163]:
from PIL import Image

# Test image path
test_image_path = "/Users/kiaracueva/Desktop/Screenshots/Screen Shot 2023-03-10 at 12.07.05 AM.png" # Replace with a valid image file path

# Open the test image
image = Image.open(test_image_path)

# Run the pipeline
caption, image_sentiment, matching_song = image_to_song_pipeline(image, ds_sample)

# Print the results
print("Generated Caption:", caption)
print("Image Sentiment:", image_sentiment)
if matching_song is not None:
    print("Matching Song:")
    print(matching_song[['title', 'artist', 'year', 'dominant_sentiment']])
else:
    print("No matching song found.")


Generated Caption: two women are smiling and posing for a picture in a restaurant
Image Sentiment: [{'label': 'joy', 'score': 0.5492992628041324}, {'label': 'neutral', 'score': 0.28157976434394966}, {'label': 'joy', 'score': 0.05477567056771794}]
Matching Song:
                   title    artist  year dominant_sentiment
709  Northern Chaos Gods  Immortal  2018            neutral
